In [1]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import (
    SimpleDocTemplate,
    Paragraph,
    Spacer,
    PageBreak,
    Image,
    ListFlowable,
    ListItem,
)
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_JUSTIFY, TA_CENTER, TA_LEFT
from reportlab.lib.colors import navy, black, whitesmoke, lightgrey


def generate_system_design_report():
    pdf_filename = "ShopAssist_AI_2.0_System_Report_Deven_Kalra.pdf"
    doc = SimpleDocTemplate(
        pdf_filename,
        pagesize=letter,
        rightMargin=72,
        leftMargin=72,
        topMargin=72,
        bottomMargin=72,
    )

    # --- Styles ---
    styles = getSampleStyleSheet()

    # Custom Styles to mimic a LaTeX academic report
    title_style = ParagraphStyle(
        name="Title",
        parent=styles["Heading1"],
        alignment=TA_CENTER,
        fontSize=24,
        spaceAfter=24,
        fontName="Times-Bold",
        textColor=navy,
    )
    subtitle_style = ParagraphStyle(
        name="Subtitle",
        parent=styles["Normal"],
        alignment=TA_CENTER,
        fontSize=12,
        spaceAfter=12,
        fontName="Times-Roman",
    )
    h1_style = ParagraphStyle(
        name="Header1",
        parent=styles["Heading2"],
        fontSize=16,
        spaceBefore=16,
        spaceAfter=10,
        fontName="Times-Bold",
        textColor=navy,
        keepWithNext=True,
    )
    h2_style = ParagraphStyle(
        name="Header2",
        parent=styles["Heading3"],
        fontSize=13,
        spaceBefore=10,
        spaceAfter=6,
        fontName="Times-Bold",
        textColor=black,
        keepWithNext=True,
    )
    normal_style = ParagraphStyle(
        name="Body",
        parent=styles["Normal"],
        alignment=TA_JUSTIFY,
        fontSize=11,
        leading=14,
        spaceAfter=10,
        fontName="Times-Roman",
    )
    code_style = ParagraphStyle(
        name="Code",
        parent=styles["Code"],
        fontName="Courier",
        fontSize=9,
        leading=11,
        backColor=whitesmoke,
        borderPadding=8,
        spaceAfter=10,
    )

    content = []

    # --- Title Page ---
    content.append(Spacer(1, 60))
    content.append(Paragraph("ShopAssist AI 2.0", title_style))
    content.append(Paragraph("System Design & Implementation Report", subtitle_style))
    content.append(Spacer(1, 40))
    content.append(Paragraph("<b>Submitted by:</b> Deven Kalra", subtitle_style))
    content.append(PageBreak())

    # --- Executive Summary ---
    content.append(Paragraph("Executive Summary", h1_style))
    intro_text = """
    ShopAssist AI 2.0 is a specialized conversational commerce agent designed to assist users in selecting laptops based on their specific needs.
    Unlike standard chatbots, ShopAssist 2.0 employs a deterministic <b>Function Calling</b> architecture that retrieves real-time product data from a CSV database.
    This hybrid approach combines the natural language understanding of GPT-4 with the factual accuracy of a structured search engine, eliminating hallucinations regarding product specifications and prices.
    """
    content.append(Paragraph(intro_text, normal_style))

    # --- 1. System Design ---
    content.append(Paragraph("1. System Architecture", h1_style))

    design_intro = """
    The system utilizes a <b>Modular Agentic Architecture</b> orchestrated by a central Dialogue Management System.
    The conversation flow is linear but guarded by multiple validation layers to ensure safety and data integrity.
    """
    content.append(Paragraph(design_intro, normal_style))

    content.append(Paragraph("<b>1.1 High-Level Data Flow</b>", h2_style))

    # Text-based diagram
    diagram = """
    [USER INPUT]
         |
         v
    [MODERATION LAYER] (OpenAI Moderation API)
         |
         v
    [DIALOGUE MANAGER] (Orchestrator)
         |
         +---> [INTENT AGENT] (Extracts: Budget, GPU, RAM, etc.)
         |          |
         |          +---(Missing Info)---> [Ask Clarifying Question]
         |          |
         |          +---(Complete)-------> [FUNCTION CALLING AGENT]
         |                                      |
         |                                      v
         |                              [TOOL: compare_laptops]
         |                                      |
         |                                      v
         |                           [DATA LAYER: Pandas/CSV]
         |                                      |
         |                                      v
    [RESPONSE GENERATOR] <--- [Validation & Recommendation]
    """
    content.append(
        Paragraph(diagram.replace("\n", "<br/>").replace(" ", "&nbsp;"), code_style)
    )

    # --- 2. Component Documentation ---
    content.append(Paragraph("2. Module Documentation", h1_style))

    modules = [
        (
            "Dialogue Management System",
            "The central loop (<code>dialogue_mgmt_system</code>) that captures user input and routes it through the various layers. It maintains the conversation history and manages the exit conditions.",
        ),
        (
            "Moderation Layer",
            "Implemented in <code>moderation_check</code>. Every user input is passed to the OpenAI Moderation API to detect harassment, violence, or self-harm before processing.",
        ),
        (
            "Intent & Slot Filling Layer",
            "Implemented in <code>intent_confirmation_layer</code>. This agent acts as a gatekeeper. It analyzes the conversation to ensure 6 key 'slots' are filled: GPU Intensity, Display Quality, Portability, Multitasking, Processing Speed, and Budget.",
        ),
        (
            "Function Execution Engine",
            "Implemented in <code>get_chat_completions_using_function_calling</code>. This specialized agent constructs a structured JSON payload to call the Python function <code>compare_laptops_with_user</code>.",
        ),
        (
            "Data Layer (Product Map)",
            "The <code>product_map_layer</code> preprocesses the raw laptop descriptions from the CSV, converting unstructured text into structured 'High/Medium/Low' tags to allow for mathematical comparison.",
        ),
    ]

    for title, desc in modules:
        content.append(Paragraph(f"<b>{title}:</b>", h2_style))
        content.append(Paragraph(desc, normal_style))

    # --- 3. Technical Implementation ---
    content.append(Paragraph("3. Technical Implementation", h1_style))

    content.append(Paragraph("<b>3.1 Function Calling & JSON Handling</b>", h2_style))
    tech_text = """
    The core innovation of ShopAssist 2.0 is the integration of OpenAI's Function Calling API.
    The system defines a strict schema for the <code>compare_laptops_with_user</code> tool.
    Crucially, the implementation handles the variability of LLM outputs by using robust parsing logic (<code>json.loads</code> and <code>ast.literal_eval</code>) to convert string responses into executable Python dictionaries.
    """
    content.append(Paragraph(tech_text, normal_style))

    content.append(Paragraph("<b>3.2 Reliability (Tenacity)</b>", h2_style))
    retry_text = """
    To mitigate API instability, the system wraps API calls with the <code>tenacity</code> library.
    The <code>@retry</code> decorator implements exponential backoff strategies, ensuring the system gracefully handles timeouts or rate limits without crashing.
    """
    content.append(Paragraph(retry_text, normal_style))

    # --- 4. Evaluation ---
    content.append(Paragraph("4. Evaluation & Challenges", h1_style))
    eval_text = """
    The notebook includes a custom evaluation metric <code>evaluate_model_response</code> which compares the LLM's extracted profile against a ground-truth tag set.
    """
    content.append(Paragraph(eval_text, normal_style))

    challenges = [
        "<b>Integrating Functional Calling:</b> Enhancing the customer experience by crafting a superior and more intelligent chatbot by integrating functional calling",
        "<b>Refining Conversation Flow::</b> Challenges were over comed by ",
    ]
    for chall in challenges:
        content.append(Paragraph(chall, normal_style, bulletText="•"))

    # --- Conclusion ---
    content.append(Paragraph("5. Conclusion", h1_style))
    conc_text = """
    ShopAssist AI 2.0 successfully demonstrates a robust implementation of Retrieval Augmented Generation (RAG) concepts using Function Calling.
     Enforcing strict intent validation, making conversation more dynamic the system provides accurate, safe, and context-aware product recommendations.
    """
    content.append(Paragraph(conc_text, normal_style))

    # Build PDF
    doc.build(content)
    print(f"PDF Generated Successfully: {pdf_filename}")


if __name__ == "__main__":
    generate_system_design_report()

PDF Generated Successfully: ShopAssist_AI_2.0_System_Report_Deven_Kalra.pdf
